In [1]:
# Includes Librarires
import lhapdf
import math
import numpy as np
from random import sample

In [2]:
# Build PDF input samples
def build_pdf(params, x_grid):
    if params['nb_replicas'] < 1:
        raise ValueError("Number of input replicas must be greater or equal to 1")
    else:
        input_size = params['nb_replicas']

    pdf_name = params["pdf_set_name"]
    flavors  = params['flavors']
    q2_scale = params['Q2_scale']

    """
    At the moment, the shape of the input pdf replicas (which gets fed to the
    network) is (nb_replicas, flavor, x_grid). This is due to the fact that,
    at the end, the generator has to generate as many replicas as on wants
    (meaning that the number of replicas has to be in the first argument).

    Preferably, one would want to have (nb_flavor, nb_replicas, x_grid).
    Maybe there are some ways to do this?
    """
    # If only a single input replicas is given
    if input_size == 1:
        # Pick the Central Value
        pdf = [lhapdf.mkPDF(pdf_name, 0)]
    else:
        pdf = sample(lhapdf.mkPDFs(pdf_name), input_size)

    # Data of Shape (pdf_replicas, flavors, x_grid)
    data = []
    for p in pdf:
        x_space = []
        for x in x_grid:
            """
            !!! Here is the crucial part !!!

            The model cannot properly learn the distribution of a given flavor
            (esp. for the gluon) as the result blows up at small-x. For some
            reasons, the model gets stuck and as a result the training does
            not improve.

            Below, the quarks are represented in terms of their valence. One
            possible way to overcome this might be to map the pdf into a much
            more manageable function and apply a back transform at the end.
            """
            x_space.append(p.xfxQ2(flavors,x,q2_scale)-p.xfxQ2(-flavors,x,q2_scale))
        data.append(x_space)
    return np.array(data)

In [3]:
%matplotlib inline 
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cm as cmx
from matplotlib.font_manager import FontProperties

In [4]:
# Plot the Result
def plot_input_replicas(data, x_grid):
    plt.figure(figsize=[9,6])
    for pdf in data:
        plt.plot(x_grid, pdf, color='red', alpha=0.45)
    plt.xlabel('x')
    plt.ylabel('xfx')
    plt.grid(True)
    plt.xlim([0,1])
    plt.ylim([-2e-1,6e-1])
    plt.title('Input_replicas')
    plt.savefig("input_replicas_flavors.png")
    

In [5]:
# Set Physical Scales
input_params = {'pdf_set_name': "NNPDF31_nnlo_as_0118", 
                'flavors': 1,
                'Q2_scale': 1.7, 
                'nb_replicas': 50}
                
# Toy x_grid
x = np.logspace(math.log(1e-5), math.log(1), num=100, base=math.exp(1))

In [6]:
input_pdf = build_pdf(input_params, x)
input_pdf.shape

(50, 100)

In [ ]:
plot_input_replicas(input_pdf, x)

### Test of different GAN Models (as implemented in *src*) for simple Input

In [ ]:
# Include libraries for Keras & Tensorflow
import os, sys
import tensorflow as  tf
import keras.backend as K
from keras import Model
from keras.models import Sequential
from keras.layers import Layer
from keras.layers import Dense, Dropout, Input
from keras.layers import Reshape
from keras.layers import Conv2D
from keras.layers import Flatten
from keras.layers import BatchNormalization
from keras.layers import LSTM
from keras import initializers
from keras.layers import Activation
from keras.optimizers import SGD
from keras.optimizers import Adam
from keras.optimizers import RMSprop
from keras.optimizers import Adadelta
from keras.layers.advanced_activations import ELU
from keras.layers.advanced_activations import ReLU
from keras.layers.advanced_activations import LeakyReLU
from keras.initializers import RandomNormal

In [ ]:
# Layer derived class for preprocessing
class preprocessing_fit(Layer):
    """
    Multiply the previous layer by:
            x^a (1-x)^b
    """

    def __init__(self, xval, trainable=True, kernel_initializer='ones', **kwargs):
        self.xval = xval
        self.trainable = trainable
        self.kernel_initializer = initializers.get(kernel_initializer)
        super(preprocessing_fit, self).__init__(**kwargs)

    def build(self, input_shape):
        self.kernel = self.add_weight(name='kernel', shape=(2,),
                initializer=self.kernel_initializer,
                trainable=self.trainable)
        super(preprocessing_fit, self).build(input_shape)

    def compute_output_shape(self, input_shape):
        return input_shape

    def call(self, pdf):
        xres = self.xval**self.kernel[0] * (1-self.xval)**self.kernel[1]
        return pdf*xres

# Custom layer that contains the information on x_grid
class xlayer(Layer):

    """
    Custom array that inputs the information on the x-grid.
    """

    def __init__(self, output_dim, xval, kernel_initializer='glorot_uniform', **kwargs):
        self.output_dim = output_dim
        self.xval = K.constant(xval)
        self.kernel_initializer = initializers.get(kernel_initializer)
        super(xlayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.kernel = self.add_weight(name='kernel', shape=(K.int_shape(self.xval)[0],
                input_shape[1], self.output_dim), initializer=self.kernel_initializer,
                trainable=True)
        super(xlayer, self).build(input_shape)

    def call(self, x):
        # xres outputs (None, input_shape[1], len(x_pdf))
        xres = K.tf.tensordot(x, self.xval, axes=0)
        # xfin outputs (None, output_dim)
        xfin = K.tf.tensordot(xres, self.kernel, axes=([1,2],[0,1]))
        return xfin

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_dim)

In [ ]:
# Define Model for Discriminator
def discriminator(params, output_size):
    # Weights initialization
    init  = RandomNormal(stddev=0.02)
    # Weights constraint
    const = None
    # Activation per layer
    d_activ = params['d_activ']
    d_nodes = params['d_nodes']

    ## Critic Architecture ##
    D_input = Input(shape=(output_size,))

    # 1st hidden dense layer
    D_1l = Dense(d_nodes, kernel_initializer=init, kernel_constraint=const)(D_input)
    D_1a = d_activ(D_1l)
    # 2nd hidden dense layer
    D_2l = Dense(d_nodes//2, kernel_initializer=init, kernel_constraint=const)(D_1a)
    D_2a = d_activ(D_2l)

    # Output 1 dimensional probability
    D_output = Dense(1, activation='sigmoid')(D_2a)

    model = Model(D_input, D_output)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    return model

# Define Model for Generator
def generator(params, output_size):
    # Weights initialization
    init  = RandomNormal(stddev=0.02)
    # Activation per layer
    g_activ = params['g_activ']
    # noise size and nodes dim
    g_nodes     = params['g_nodes']
    noise_size  = params['noise_size']

    ## Generator Architecture ##
    # Input of G/Random noise of 1 dim vector
    G_input = Input(shape=(noise_size,))

    # 1st hidden dense layer
    G_1l = Dense(g_nodes//4, kernel_initializer=init)(G_input)
    G_1a = g_activ(G_1l)
    if params['x_multiply']:
        G_2l = xlayer(g_nodes//2, params['x_grid'], kernel_initializer=init)(G_1a)
    else:
        G_2l = Dense(g_nodes//2, kernel_initializer=init)(G_1a)
    G_2a = g_activ(G_2l)
    G_3l = Dense(g_nodes, kernel_initializer=init)(G_2a)
    G_3a = g_activ(G_3l)
    G_4l = Dense(g_nodes*2, kernel_initializer=init)(G_3a)
    G_4a = g_activ(G_4l)
    G_5l = Dense(output_size, activation='tanh', kernel_initializer=init)(G_4a)

    # Output from the generator
    G_output = G_5l
    if params['preprocessing']:
        G_output = preprocessing_fit(params['x_grid'])(G_5l)

    model = Model(G_input, G_output)

    return model 

In [ ]:
# Define Adversarial Model
def adversarial(generator, discriminator):
    # make weights in the discriminator not trainable
    discriminator.trainable = False

    model = Sequential()
    # add generator
    model.add(generator)
    # add the discriminator
    model.add(discriminator)
    # compile model
    model.compile(loss='binary_crossentropy', optimizer='adam')

    return model

In [ ]:
model_params = {'g_nodes': 128, 'g_activ': LeakyReLU(0.2), 'noise_size': 100,
                'd_nodes': 128, 'd_activ': LeakyReLU(0.2),
                'x_multiply': True, 'preprocessing': True, 'x_grid': x}

In [ ]:
# Sample input replicas
def sample_input_replicas(data, batch_size):
    """ This is a bit trickier as one wants to batch only a subset
    of the input replicas while the input data has the following
    shape (nb_flavors, nb_replicas, x_grid)"""

    index_replicas = np.random.randint(0, data.shape[0], batch_size)
    pdf_batch = data[index_replicas]
    # Create labels of ones for reals
    y_real = np.ones(batch_size)
    return pdf_batch, y_real

# generate points in latent space as input for the generator
def latent_space(batch_size, noise_size):
    # generate points in the latent space
    noise_latent = np.random.randn(noise_size * batch_size)
    # reshape into a batch of inputs for the network
    return noise_latent.reshape(batch_size, noise_size)

# Sample output fake replicas
def generate_fake_replicas(generator, batch_size, noise_size):
    # Feed noise to the generator
    noise    = latent_space(batch_size, noise_size)
    pdf_fake = generator.predict(noise)
    # Create labels of 0 for fake samples
    y_fake   = np.zeros(batch_size)
    return pdf_fake, y_fake

# Sample noise that will enter in the Adv. GAN
def sample_input_noise(batch_size, noise_size):
    noise = latent_space(batch_size, noise_size)
    # Create labels of ones
    y_gen = np.ones(batch_size)
    return noise, y_gen

In [ ]:
# Define plot function to summarize the performance
def plot_generated_pdf(discriminator, generator, data,
                       batch_size, x_grid, noise_size, kth):
    path = os.getcwd()
    # Create folder
    if not os.path.exists('%s/iterations' % path):
        try:
            original_umask = os.umask(000)
            os.makedirs('%s/iterations' % path, 0o777)
        finally:
            os.umask(original_umask)
    else:
        pass
    
    # Prepare real samples
    x_real, y_real = sample_input_replicas(data, batch_size)
    # Evaluate discriminator on real samples
    _, acc_real = discriminator.evaluate(x_real, y_real, verbose=0)
    # Prepare fake samples
    x_fake, y_fake = generate_fake_replicas(generator, batch_size, noise_size)
    # Evaluate discriminator on fake samples
    _, acc_fake = discriminator.evaluate(x_fake, y_fake, verbose=0)

    plt.figure(figsize=[9,6])
    for real, fake in zip(x_real, x_fake):
        plt.plot(x_grid, real, color='red', label='real', alpha=0.45)
        plt.plot(x_grid, fake, color='blue', label='fake', alpha=0.45)
    plt.xlabel('x')
    plt.ylabel('xfx')
    plt.grid(True)
    plt.xlim([0,1])
    plt.ylim([-2e-1,2])
    plt.title('real_vs_fake_replicas')
    plt.savefig("%s/iterations/plot_at_iteration_%d.png" %(path, kth))

In [ ]:
# Train the Model
def train_gan(disc_model, gen_model, adv_model, x_grid,
              data, noise, nb_epochs=100, batch_size=1):
    # Batches per epoch
    batch_per_epoch = int(data.shape[1]/batch_size)
    # Training per iteration
    nb_steps = batch_per_epoch * nb_epochs
    # Compute half-batch
    if batch_size < 2:
        half_batch_size = 1
    else:
        half_batch_size = int(batch_size/2)

    # Training iterations
    for tr in range(1, nb_steps+1):

        # Train the discriminator for a certain steps
        for _ in range(1):
            # Train on real samples
            x_real, y_real = sample_input_replicas(data, half_batch_size)
            r_dloss, r_dacc = disc_model.train_on_batch(x_real, y_real)
            # Train on fake samples
            x_fake, y_fake = generate_fake_replicas(gen_model, half_batch_size, noise)
            f_dloss, f_dacc = disc_model.train_on_batch(x_fake, y_fake)
        
        # Train the Adversarial GAN for a certain setps
        for _ in range(1):
            # Generate latent point for the generator
            inp_noise, y_gen = sample_input_noise(batch_size, noise)
            gan_loss = adv_model.train_on_batch(inp_noise, y_gen)
        
        # Logoutput
        if tr % 10000 == 0:
            print("Iter:{} out of {}. disc_loss: {:6f}. gan_loss: {:6f}"
                   .format(tr, nb_steps+1, r_dloss+f_dloss, gan_loss))
            # Generates Plot
            plot_generated_pdf(disc_model, gen_model, data, batch_size, x_grid, noise, tr)

In [ ]:
# Define output size
output_size = x.shape[0]

In [ ]:
# Disc
disc_model = discriminator(model_params, output_size)
disc_model.summary()

In [ ]:
# Gen
gen_model = generator(model_params, output_size)
gen_model.summary()

In [ ]:
# Adversarial
gan_model = adversarial(gen_model, disc_model)
gan_model.summary()

In [ ]:
train_gan(disc_model, gen_model, gan_model, x, input_pdf, model_params['noise_size'], nb_epochs=1000, batch_size=1)

## ERF Checks

In [7]:
import sys
sys.path.append('../src/wganpdfs')
from custom import normalizationK
from custom import smm

In [8]:
test_params  = {'pdf_set_name': "NNPDF30_nnlo_as_0118", 
                'flavors': 1,
                'Q2_scale': 1.7, 
                'nb_replicas': 50}

In [9]:
nnpdf30 = build_pdf(test_params, x)
nnpdf31 = build_pdf(input_params, x)

In [10]:
test_cfd68 = normalizationK(nnpdf31, nnpdf30, 1000)
# Take randomized sets from the true
rd_nnpdf30 = test_cfd68.random_replicas(40)

In [11]:
tr, fk = test_cfd68.cfd68('mean', rd_nnpdf30)

In [12]:
test_cfd68.Nk_mean()

2.330885355201029e-30

In [13]:
estimators = {'estimators': ['mean', 'stdev']}

In [14]:
smetric = smm(nnpdf30, nnpdf31, estimators)

In [15]:
smetric.ERF()

8.808244437939416e+31